In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# Move model to GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

/Users/prudhvivuda/.virtualenvs/genai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(105879, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [2]:
from torch.nn.functional import softmax

def predict_sentiment(text):
    # Tokenize input
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        return_token_type_ids=False,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )

    # Move inputs to device
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    # Predict
    model.eval()
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        probabilities = softmax(logits, dim=1)
        predicted_label = torch.argmax(probabilities, dim=1).item() + 1  # Add 1 to convert 0-indexed to 1-5 scale

    # Map label to sentiment
    sentiment_map = {
        1: "Extremely Negative",
        2: "Negative",
        3: "Neutral",
        4: "Positive",
        5: "Extremely Positive"
    }
    return sentiment_map[predicted_label]

In [3]:
# Sample inputs
sample_texts = [
    "I am so excited about this project",
    "I feel really anxious about the future.",
    "It's just another ordinary day.",
    "I want to suicide"
]

# Predict sentiments
for text in sample_texts:
    sentiment = predict_sentiment(text)
    print(f"Text: '{text}' -> Sentiment: {sentiment}")

Text: 'I am so excited about this project' -> Sentiment: Extremely Negative
Text: 'I feel really anxious about the future.' -> Sentiment: Neutral
Text: 'It's just another ordinary day.' -> Sentiment: Extremely Positive
Text: 'I want to suicide' -> Sentiment: Extremely Negative
